In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [45]:
# 데이터 불러오기
data = pd.read_csv('merged_predict2.csv')

# 피처와 타겟 변수 설정
features = data[['검색건수', '관외 이동량', '숙박업소 개수','sns']]
target = data['방문자수']


In [46]:
# 데이터셋 분할 (훈련셋과 테스트셋)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 생성
model = RandomForestRegressor(n_estimators=100, random_state=42)

# 모델 학습
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [47]:
# 예측 및 성능 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# 정확도 계산 및 출력
def calculate_accuracy(r2):
    return max(0, min(100, r2 * 100))  # 0-100 사이로 제한

rf_accuracy = calculate_accuracy(r2)

print(f'Mean Squared Error: {mse}')
print("Mean Absolute Error (MAE):", mae)
print(f'R-squared: {r2}')
print('Accuracy:',rf_accuracy)

Mean Squared Error: 515798428050.42896
Mean Absolute Error (MAE): 437062.69509043935
R-squared: 0.9537876451844897
Accuracy: 95.37876451844896


In [48]:
predict_data = data[data['일시'].isin([1, 2, 3, 4, 5])]
predict_features = predict_data[['검색건수', '관외 이동량', '숙박업소 개수','sns']]

# 2024년 1월부터 5월까지의 방문자수 예측
predicted_visitors = model.predict(predict_features)

# 결과 출력
predict_data['예측 방문자수'] = predicted_visitors

C:\Users\lotte\AppData\Local\Temp\ipykernel_15516\1769174731.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['예측 방문자수'] = predicted_visitors


In [49]:
predict_data

,일시,지역명,방문자수,검색건수,관외 이동량,숙박업소 개수,sns,예측 방문자수
0,1,강원특별자치도 춘천시,8979957,229813,444582,312,39944.0,8842602.99
1,1,강원특별자치도 원주시,9530785,199254,482623,133,32109.0,9163476.77
2,1,강원특별자치도 강릉시,8031760,452294,565563,658,65556.0,8002811.29
3,1,강원특별자치도 동해시,2946561,64643,167985,101,5425.0,2917294.26
4,1,강원특별자치도 태백시,1491579,32726,92434,20,13153.0,1373307.00
...,...,...,...,...,...,...,...,...
1829,5,충청북도 증평군,1388821,29203,92801,6,8073.0,1298698.27
1830,5,충청북도 진천군,2551282,84387,260404,39,11037.0,2773009.44
1831,5,충청북도 괴산군,1536345,29203,158593,74,11518.0,1521929.91
1832,5,충청북도 음성군,3975691,78947,317368,29,12392.0,3795632.67


In [50]:
real01 = pd.read_csv('202401방문자수.csv',encoding='euc-kr')
real02 = pd.read_csv('202402방문자수.csv',encoding='euc-kr')
real03 = pd.read_csv('202403방문자수.csv',encoding='euc-kr')
real04 = pd.read_csv('202404방문자수.csv',encoding='euc-kr')
real05 = pd.read_csv('202405방문자수.csv',encoding='euc-kr')

In [51]:
real01

,일시,지역명,시군구,방문자 수
0,1,강원특별자치도,고성군,1452663.0
1,1,강원특별자치도,춘천시,9024519.0
2,1,강원특별자치도,원주시,9956163.0
3,1,강원특별자치도,강릉시,8218107.0
4,1,강원특별자치도,동해시,2874821.0
...,...,...,...,...
248,1,충청북도,단양군,1047031.0
249,1,충청북도,청주시 상당구,5048982.0
250,1,충청북도,청주시 서원구,4193097.0
251,1,충청북도,청주시 흥덕구,6715304.0


In [52]:
real_data = pd.concat([real01, real02, real03, real04, real05])

# '지역명' 컬럼 생성 (지역명과 시군구를 합쳐서)
real_data['지역'] = real_data['지역명'] + ' ' + real_data['시군구']

# 불필요한 컬럼 삭제
real_data.drop(columns=['지역명', '시군구'], inplace=True)

In [53]:
real_data = real_data.rename(columns={'지역': '지역명'})

In [54]:
real_data

,일시,방문자 수,지역명
0,1,1452663.0,강원특별자치도 고성군
1,1,9024519.0,강원특별자치도 춘천시
2,1,9956163.0,강원특별자치도 원주시
3,1,8218107.0,강원특별자치도 강릉시
4,1,2874821.0,강원특별자치도 동해시
...,...,...,...
248,5,15900000.0,충청북도 증평군
249,5,29900000.0,충청북도 진천군
250,5,17900000.0,충청북도 괴산군
251,5,40500000.0,충청북도 음성군


In [55]:
merged_df = pd.merge(real_data, predict_data, on=['일시', '지역명'])

In [56]:
merged_df=merged_df[['일시','지역명','방문자 수','예측 방문자수']]

In [57]:
merged_df

,일시,지역명,방문자 수,예측 방문자수
0,1,강원특별자치도 고성군,1452663.0,1612621.51
1,1,강원특별자치도 춘천시,9024519.0,8842602.99
2,1,강원특별자치도 원주시,9956163.0,9163476.77
3,1,강원특별자치도 강릉시,8218107.0,8002811.29
4,1,강원특별자치도 동해시,2874821.0,2917294.26
...,...,...,...,...
754,5,충청북도 증평군,15900000.0,1298698.27
755,5,충청북도 진천군,29900000.0,2773009.44
756,5,충청북도 괴산군,17900000.0,1521929.91
757,5,충청북도 음성군,40500000.0,3795632.67


In [58]:
# 방문자 수와 예측 방문자 수의 차이 계산
merged_df['오차'] = merged_df['방문자 수'] - merged_df['예측 방문자수']

# MSE 계산
mse = mean_squared_error(merged_df['방문자 수'], merged_df['예측 방문자수'])

print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 755206010985400.6


In [59]:
merged_df

,일시,지역명,방문자 수,예측 방문자수,오차
0,1,강원특별자치도 고성군,1452663.0,1612621.51,-159958.51
1,1,강원특별자치도 춘천시,9024519.0,8842602.99,181916.01
2,1,강원특별자치도 원주시,9956163.0,9163476.77,792686.23
3,1,강원특별자치도 강릉시,8218107.0,8002811.29,215295.71
4,1,강원특별자치도 동해시,2874821.0,2917294.26,-42473.26
...,...,...,...,...,...
754,5,충청북도 증평군,15900000.0,1298698.27,14601301.73
755,5,충청북도 진천군,29900000.0,2773009.44,27126990.56
756,5,충청북도 괴산군,17900000.0,1521929.91,16378070.09
757,5,충청북도 음성군,40500000.0,3795632.67,36704367.33


In [60]:
importance = model.feature_importances_

# 결과 출력
print("Feature Importance:")
for i, feat in enumerate(features.columns):
    print(feat, ':', importance[i])

# 예측 결과
predictions = model.predict(features)
print("Predictions:", predictions)

Feature Importance:
검색건수 : 0.07489778773637598
관외 이동량 : 0.8176664780263411
숙박업소 개수 : 0.06876131548363201
sns : 0.03867441875365074
Predictions: [8842602.99 9163476.77 8002811.29 ... 1122641.78 3167613.3  1107435.28]


### 하이퍼 파라미터 튜닝 이용해 보자 -> 그리드 서치

In [61]:
# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 랜덤 포레스트 모델 생성
model = RandomForestRegressor(random_state=42)

# 그리드 서치 설정
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# 그리드 서치를 사용하여 모델 학습
grid_search.fit(X_train_scaled, y_train)

# 최적의 하이퍼파라미터 출력
print(f'Best Parameters: {grid_search.best_params_}')

# 최적의 모델로 예측 및 성능 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 정확도 계산 및 출력
def calculate_accuracy(r2):
    return max(0, min(100, r2 * 100))  # 0-100 사이로 제한

rf_accuracy = calculate_accuracy(r2)

print(f'Mean Squared Error: {mse}')
print("Mean Absolute Error (MAE):", mae)
print(f'R-squared: {r2}')
print('Accuracy:', rf_accuracy)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 532990370633.0028
Mean Absolute Error (MAE): 450325.7391705384
R-squared: 0.9522473532654222
Accuracy: 95.22473532654222


### 스케일링 + 그리드 서치

In [72]:
# 파이프라인 설정 (스케일링 + 모델)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(random_state=42))
])

# 그리드 서치를 위한 하이퍼파라미터 설정
param_grid = {
    'rf__n_estimators': [100, 200, 300],
    'rf__max_depth': [10, 20, 30, None],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['sqrt', 'log2']
}

# 그리드 서치 객체 생성
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, 
                           scoring='r2', verbose=1, n_jobs=-1)

# 그리드 서치로 모델 학습
grid_search.fit(X_train, y_train)

# 최적의 모델 출력
print("Best Parameters:", grid_search.best_params_)

# 최적의 모델로 예측 및 성능 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# 정확도 계산 및 출력
def calculate_accuracy(r2):
    return max(0, min(100, r2 * 100))  # 0-100 사이로 제한

rf_accuracy = calculate_accuracy(r2)

print(f'Mean Squared Error: {mse}')
print("Mean Absolute Error (MAE):", mae)
print(f'R-squared: {r2}')
print('Accuracy:', rf_accuracy)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters: {'rf__max_depth': 30, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}
Mean Squared Error: 528013106261.6215
Mean Absolute Error (MAE): 450057.6233204135
R-squared: 0.952693285425414
Accuracy: 95.2693285425414
